In [15]:
import wandb
import ultralytics
from ultralytics import YOLO
import torch
import cv2
import pandas as pd
import torch
import os
import glob

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True" #to help with memory allocation issues(weak gpu)
torch.cuda.empty_cache() #weak gpu
# check yolo version and using cuda
print(ultralytics.__version__, torch.cuda.is_available())

8.3.62 True


In [3]:
!nvidia-smi

Fri Feb  7 18:30:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  |   00000000:06:00.0  On |                  N/A |
| 32%   47C    P0             29W /  125W |     490MiB /   6144MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
wandb.login()

Arguments setting for the model

In [4]:
args = dict(
    data = "data.yaml",
    epochs = 20,
    patience = 5,
    batch = 4,
    save = True,
    save_period = 5,
    device = '0',
    workers = 1,
    project = "weights",
    name = "try",
    exist_ok = True, #overwrite the existing run for iterative testing, turn false when confident
    pretrained = True,
    optimizer = 'auto',
    resume = False,
    amp = False, #mixed precision training for faster training, might cause some issues
    fraction = 0.1, #fraction of dataset if resouce limited, change to 1 when confident
    profile = False, #enable ONNX and TensorRT profiling
    freeze = None, #freeze the first N layers or can specify the layers by index
    #for validation part
    conf = 0.01,
    iou = 0.6,
    degrees = 0.0, 
    shear = 0.0,
    perspective = 0.0,
    plots = True,
    half = True, #use half precision for weaker gpu
)

Loading the model yoloV8 and running wandb for logging

In [ ]:
wandb.init(project='yolov8xGlottis', job_type='training, config=args', settings=wandb.Settings(init_timeout=500))

Training of model

In [ ]:
#training model
model = YOLO('yolov8x.pt')


In [8]:
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.3.73 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.62  Python-3.9.21 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=20, time=None, patience=5, batch=4, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=1, project=weights, name=try, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=0.1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.01, iou=0.6, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

train: Scanning D:\FYP Datasets\fyp\training\train\labels.cache... 4740 images, 296 backgrounds, 0 corrupt: 100%|██████████| 4740/4740 [00:00<?, ?it/s]
val: Scanning D:\FYP Datasets\fyp\training\val\labels.cache... 10665 images, 696 backgrounds, 0 corrupt: 100%|██████████| 10665/10665 [00:00<?, ?it/s]


Plotting labels to weights\try\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to weights\try
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      8.38G      1.443       2.35      1.194         10        640: 100%|██████████| 1185/1185 [36:48<00:00,  1.86s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [4:04:59<00:00, 11.02s/it]  

                   all      10665       9969      0.812      0.683      0.761      0.504


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      3.72G     0.7645     0.4583     0.9309          2        640: 100%|██████████| 1185/1185 [01:49<00:00, 10.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:17<00:00, 17.21it/s]


                   all      10665       9969      0.901      0.872      0.899      0.732

20 epochs completed in 5.717 hours.
Optimizer stripped from weights\try\weights\last.pt, 6.2MB
Optimizer stripped from weights\try\weights\best.pt, 6.2MB

Validating weights\try\weights\best.pt...
Ultralytics 8.3.62  Python-3.9.21 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
Model summary (fused): 168 layers, 3,006,818 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:07<00:00, 19.75it/s]


                   all      10665       9969      0.901      0.872      0.899      0.732
               Glottis       9969       9969      0.901      0.872      0.899      0.732
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to weights\try


Running Predictions on model using bagls video and dhs video

note we do not need to use openCV for video processing since we already have the video, this will not be true once we start using the webcamera

In [30]:
dhs_video = r'D:\FYP Code\testfiles\dhs.mp4'
baglsC_video = r'D:\FYP Code\testfiles\bagls-color.mp4'
bagls_video = r'D:\FYP Code\testfiles\bagls.mp4'
test_path = r'D:\FYP Datasets\fyp\training\test\images'
weight_path = r'weights\try\weights\best.pt'

In [31]:
model = YOLO(weight_path)
# model.predict(source= test_path, save_dir = 'results', save= True)
model.predict(source=dhs_video, save_dir = 'results', save= True)
model.predict(source=baglsC_video, save_dir = 'results', save= True)
model.predict(source=bagls_video, save_dir = 'results', save= True)




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 7.2ms
video 1/1 (frame 2/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 8.2ms
video 1/1 (frame 3/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 6.4ms
video 1/1 (frame 4/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 6.1ms
video 1/1 (frame 5/808) D:\FYP Code\testfiles\dhs.mp4: 640x640 (no detections), 3.4ms
video 1/1 (frame 6/808) D:\F

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Torus Tobarius', 1: 'Soft palate', 2: 'Uvula', 3: 'Glottis', 4: 'True vocal folds', 5: 'Epiglottis'}
 obb: None
 orig_img: array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [1, 1, 1],
         [2, 2, 2],
         [2, 2, 2]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [1, 1, 1],
         [0, 0, 0],
         [1, 1, 1]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [1, 1, 1],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
     

In [ ]:
# wandb.log({"final_mAP": results.metrics.mAP50})
# wandb.finish()